In [ ]:
import tensorflow as tf 
import numpy as np
import torch
import math
import os
import pandas as pd
import matplotlib.pyplot as plt 
from PIL import Image
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, r2_score

In [ ]:
from sklearn.cluster import KMeans


In [ ]:
#OP Model with L1 norm
from tensorflow.python.ops.linalg_ops import norm_v2
class OP_main():
    
    def __init__(self,Data, lam=1/np.sqrt(n1),th=0.000000000000001, rho=1.5, mup=1.25):
        
        #(Data, lam,th, rho, mup)=value_sets


        n1,n2=np.shape(Data)
        self.lam=lam #1/np.sqrt(n1)
        self.eps=0.000000000000001
        self.d_nrom=norm_v2(Data)
        norm_two=np.linalg.svd(Data,compute_uv=False)[0]#norm_v2(Data,2)
        #print(norm_two)
        norm_inf=norm_v2(Data,np.inf)/self.lam
        #print(norm_inf)
        dual_norm=np.max([norm_two, norm_inf])
        Lambda_int=Data/dual_norm
        #print(Lambda_int[0,0]) 
        self.mu=mup/norm_two #1.25
        self.maxmu=self.mu*10**(7)
        self.rho=rho #1.5
        self.stages=200 
        self.tol=10**(-7) 
        
        
        X_int, E_int= tf.zeros([n1,n2], tf.float64), tf.zeros([n1,n2], tf.float64)
        value_sets=(Data, X_int, E_int, Lambda_int)
        self.FX, self.FE = self.OP_ADMM_main(value_sets) 
        self.Elabel=self.getLable(self.FE,th)
 
    def OP_ADMM_main(self, inputs):
        (Data, X_int, E_int, Lambda_int) = inputs  
        self.X,self.E,self.Lambda  = [],[],[]  
        self.E.append(self.Updata_E(Data, X_int, Lambda_int))  
        self.X.append(self.Updata_X(Data, self.E[-1], Lambda_int))      
        self.Lambda.append(self.Updata_Lambda(Data, self.X[-1], self.E[-1], Lambda_int))  
        self.mu=np.min([self.mu*self.rho, self.maxmu]) 
        for stage in range(1,self.stages): 
            self.E.append(self.Updata_E(Data, self.X[-1], self.Lambda[-1]))  
            self.X.append(self.Updata_X(Data, self.E[-1], self.Lambda[-1]))      
            self.Lambda.append(self.Updata_Lambda(Data, self.X[-1], self.E[-1], self.Lambda[-1]))   
            self.mu=np.min([self.mu*self.rho, self.maxmu]) 
            stopCriterion= norm_v2(Data-self.X[-1]-self.E[-1])/self.d_nrom 
            print("Stop Criterion: {}".format(stopCriterion)) 
            if stopCriterion < self.tol:
                break 
        return self.X[-1], self.E[-1]   
     

     

    def Updata_X(self,Data, E, Lambda):
        
        temp=-E+Data+Lambda/self.mu  
        return self.SVT(temp,1/self.mu)

 
   
    def SVT(self,X,th):
        n1, n2= np.shape(X)
        rank=np.min([n1,n2])  
        U_int, S_int, VT_int=np.linalg.svd(X, full_matrices=True, compute_uv=True, hermitian=False)
        S_int=tf.nn.relu(S_int-tf.multiply(th,tf.ones(np.shape(S_int), tf.float64)))
        rank=len(np.where(S_int > self.eps)[0])
        if rank==0:
            rank=1  
        S_int=tf.compat.v1.matrix_diag(S_int) 
        Ak_int=tf.matmul(tf.matmul(U_int[:,0:rank],S_int[0:rank,0:rank]),VT_int[0:rank,:]) 
          
        return Ak_int    
 
    def Updata_E(self, Data, X, Lambda):
      
        Temp= Data-X+Lambda/self.mu
        return self.sparse_col(  Temp, self.lam/self.mu)
 

   

    def sparse_col(self, E, tau): 
        result=[]
        n1,n2=np.shape(E)
        for i in range(n2):
            result.append(tf.nn.relu(tf.norm(E[:,i])-tau)*E[:,i]/(tf.norm(E[:,i])+self.eps)) 
        return tf.transpose(result,[1,0])

  
     

    def Updata_Lambda(self, Data, X, E, Lambda):
        Lambda= Lambda+tf.multiply(self.mu, Data-X-E)
        return Lambda
 

    def getLable(self, E,th): 
        Elabel=[]
        n1,n2=np.shape(E)
        for i in range(n2):
            if tf.norm(E[:,i])>th:
                Elabel.append(1)
            else:
                Elabel.append(0)
        return Elabel
         


In [ ]:
# test sample
Data=np.array([[7,1,2],[5,2,4],[0,3,6]], np.float64)
n1,n2=np.shape(Data)
lam=1.3 #1/np.sqrt(n1)
th=0.0000001
rho=1.5
mup=1.25
H=OP_main(Data,lam,th,rho,mup)
print(H.Elabel)
print(H.FE)

Stop Criterion: 0.02771651236316883
Stop Criterion: 0.017384501214288788
Stop Criterion: 0.004953136813105215
Stop Criterion: 0.0012521199148616802
Stop Criterion: 0.0002414259085647203
Stop Criterion: 3.704678373148938e-05
Stop Criterion: 7.225579794613062e-06
Stop Criterion: 2.2373322264342416e-06
Stop Criterion: 8.001266062565949e-07
Stop Criterion: 3.0496679119347266e-07
Stop Criterion: 1.2169189011155522e-07
Stop Criterion: 5.02008332645738e-08
[1, 0, 0]
tf.Tensor(
[[ 0.79596435  0.          0.        ]
 [ 0.4636718   0.          0.        ]
 [-0.24470637  0.          0.        ]], shape=(3, 3), dtype=float64)


In [ ]:
#OP model with GST
from tensorflow.python.ops.linalg_ops import norm_v2
class OPGST_main():
    
    def __init__(self, Data, lam=1/np.sqrt(n1),th=0.000000000000001, rho=1.5, mup=1.25,p=1):
        
         

        self.p=p
        n1,n2=np.shape(Data)
        self.lam=lam  
        self.eps=0.000000000000001
        self.d_nrom=norm_v2(Data)
        norm_two=np.linalg.svd(Data,compute_uv=False)[0] 
        norm_inf=norm_v2(Data,np.inf)/self.lam 
        dual_norm=np.max([norm_two, norm_inf])
        Lambda_int=Data/dual_norm 
        self.mu=mup/norm_two #1.25
        self.maxmu=self.mu*10**(7)
        self.rho=rho  
        self.stages=200 
        self.tol=10**(-7) 
        
        X_int, E_int= tf.zeros([n1,n2], tf.float64), tf.zeros([n1,n2], tf.float64)
        value_sets=(Data, X_int, E_int, Lambda_int)
        self.FX, self.FE = self.OP_ADMM_main(value_sets) 
        self.Elabel=self.getLable(self.FE,th)
 
    def OP_ADMM_main(self, inputs):
        (Data, X_int, E_int, Lambda_int) = inputs  
        self.X,self.E,self.Lambda  = [],[],[]  
        self.E.append(self.Updata_E(Data, X_int, Lambda_int))  
        self.X.append(self.Updata_X(Data, self.E[-1], Lambda_int))      
        self.Lambda.append(self.Updata_Lambda(Data, self.X[-1], self.E[-1], Lambda_int))  
        self.mu=np.min([self.mu*self.rho, self.maxmu])   
        for stage in range(1,self.stages): 
            self.E.append(self.Updata_E(Data, self.X[-1], self.Lambda[-1]))  
            self.X.append(self.Updata_X(Data, self.E[-1], self.Lambda[-1]))      
            self.Lambda.append(self.Updata_Lambda(Data, self.X[-1], self.E[-1], self.Lambda[-1]))   
            self.mu=np.min([self.mu*self.rho, self.maxmu]) 
            stopCriterion= norm_v2(Data-self.X[-1]-self.E[-1])/self.d_nrom 
            print("Stop Criterion: {}".format(stopCriterion)) 
            if stopCriterion < self.tol:
                break 
        return self.X[-1], self.E[-1]   
     

     

    def Updata_X(self,Data, E, Lambda): 
      
        temp=-E+Data+Lambda/self.mu  
        return self.SVT(temp,1/self.mu)

 
   
    def SVT(self,X,th):
        n1, n2= np.shape(X)
        rank=np.min([n1,n2])  
        U_int, S_int, VT_int=np.linalg.svd(X, full_matrices=True, compute_uv=True, hermitian=False)
        S_int=self.Generalized_Soft_Thresholding(S_int,th,self.p)
        rank=len(np.where(S_int > self.eps)[0])
        if rank==0:
            rank=1  
        S_int=tf.compat.v1.matrix_diag(S_int) 
        Ak_int=tf.matmul(tf.matmul(U_int[:,0:rank],S_int[0:rank,0:rank]),VT_int[0:rank,:]) 
          
        return Ak_int    
 
    def Updata_E(self, Data, X, Lambda):
        
        Temp= Data-X+Lambda/self.mu
        return self.sparse_col(  Temp, self.lam/self.mu)
 

   

    def sparse_col(self, E, tau): 
        result=[]
        n1,n2=np.shape(E)
        for i in range(n2):
            result.append(self.Generalized_Soft_Thresholding(tf.norm(E[:,i]),tau,self.p)*E[:,i]/(tf.norm(E[:,i])+self.eps)) 
        return tf.transpose(result,[1,0])

  
     

    def Updata_Lambda(self, Data, X, E, Lambda):
        Lambda= Lambda+tf.multiply(self.mu, Data-X-E)
        return Lambda
 

    def getLable(self, E,th): 
        Elabel=[]
        n1,n2=np.shape(E)
        for i in range(n2):
            if tf.norm(E[:,i])>th:
                Elabel.append(1)
            else:
                Elabel.append(0)
        return Elabel
         
        
    def Generalized_Soft_Thresholding(self,S,weight,p):
        #diagS: a vector
        #weight: a value
        if np.shape(S) == ():
            diagS=[S]
        else:
            diagS=S 
        J=5 
        sigma0   =    np.abs(diagS)
        tau_GST  =   (2*weight*(1-p))**(1/(2-p))   +   p*weight*(2*(1-p)*weight)**((p-1)/(2-p)) 
        Delta=[];
        for i in range(len(diagS)):
                if sigma0[i]>tau_GST: 
                    delta=sigma0[i]  
                    for k in range(J): 
                            delta=sigma0[i]-weight*p*delta**(p-1)           
                            k=k+1  
                    Delta.append(np.sign(diagS[i])*delta)  
                else:
                    Delta.append(0) 
        if np.shape(S) == ():
            return np.array(Delta[0],np.float64) 
        else:
            return np.array(Delta,np.float64) 


    
 

In [ ]:
# test sample
Data=np.array([[7,1,2],[5,2,4],[0,3,6]], np.float64)
n1,n2=np.shape(Data)
lam=1.1 #1/np.sqrt(n1)
th=0.0000001
rho=1.5
mup=1.25
p=0.8
#value_sets=(Data,lam,th,rho,mup,p)
HGST=OPGST_main(Data,lam,th,rho,mup,p)
print(HGST.Elabel)
print(HGST.FE)

Stop Criterion: 0.06968655348307115
Stop Criterion: 0.033549524337254284
Stop Criterion: 0.014605080229469644
Stop Criterion: 0.0064766240023816065
Stop Criterion: 0.003569636981640282
Stop Criterion: 0.0013102399929058853
Stop Criterion: 0.00015949477959337335
Stop Criterion: 4.1360014467796234e-05
Stop Criterion: 1.8868610062230848e-05
Stop Criterion: 5.632345409268264e-06
Stop Criterion: 2.2922222900111563e-06
Stop Criterion: 1.0333381822169081e-06
Stop Criterion: 4.706433338958856e-07
Stop Criterion: 2.0817622293720724e-07
Stop Criterion: 9.149778874305399e-08
[1, 0, 0]
tf.Tensor(
[[ 6.38093858 -0.          0.        ]
 [ 3.76187763  0.          0.        ]
 [-1.85718315  0.          0.        ]], shape=(3, 3), dtype=float64)


In [ ]:
#OP model with ETP
from tensorflow.python.ops.linalg_ops import norm_v2
class OPETP_main():
    
    def __init__(self, Data, lam=1/np.sqrt(n1),th=0.000000000000001, rho=1.5, mup=1.25,p=1,Maxiter=1000,tol=10**(-7)):
        
       

        self.p=p
        n1,n2=np.shape(Data)
        self.lam=lam  
        self.eps=0.000000000000001
        self.d_nrom=norm_v2(Data)
        norm_two=np.linalg.svd(Data,compute_uv=False)[0] 
        norm_inf=norm_v2(Data,np.inf)/self.lam 
        dual_norm=np.max([norm_two, norm_inf])
        Lambda_int=Data/dual_norm 
        self.mu=mup/norm_two  
        self.maxmu=self.mu*10**(7)
        self.rho=rho #1.5
        self.stages=200
        self.tol=10**(-7) 
        
        X_int, E_int= tf.zeros([n1,n2], tf.float64), tf.zeros([n1,n2], tf.float64)
        value_sets=(Data, X_int, E_int, Lambda_int)
        self.FX, self.FE = self.OP_ADMM_main(value_sets) 
        self.Elabel=self.getLable(self.FE,th)
 
    def OP_ADMM_main(self, inputs):
        (Data, X_int, E_int, Lambda_int) = inputs  
        self.X,self.E,self.Lambda  = [],[],[]  
        self.E.append(self.Updata_E(Data, X_int, Lambda_int))  
        self.X.append(self.Updata_X(Data, self.E[-1], Lambda_int))      
        self.Lambda.append(self.Updata_Lambda(Data, self.X[-1], self.E[-1], Lambda_int))  
        self.mu=np.min([self.mu*self.rho, self.maxmu])   
        for stage in range(1,self.stages): 
            self.E.append(self.Updata_E(Data, self.X[-1], self.Lambda[-1]))  
            self.X.append(self.Updata_X(Data, self.E[-1], self.Lambda[-1]))      
            self.Lambda.append(self.Updata_Lambda(Data, self.X[-1], self.E[-1], self.Lambda[-1]))   
            self.mu=np.min([self.mu*self.rho, self.maxmu]) 
            stopCriterion= norm_v2(Data-self.X[-1]-self.E[-1])/self.d_nrom 
            print("Stop Criterion: {}".format(stopCriterion)) 
            if stopCriterion < self.tol:
                break 
        return self.X[-1], self.E[-1]   
     

     

    def Updata_X(self,Data, E, Lambda): 
      
        temp=-E+Data+Lambda/self.mu  
        return self.SVT(temp,1/self.mu)

 
   
    def SVT(self,X,th):
        n1, n2= np.shape(X)
        rank=np.min([n1,n2])  
        U_int, S_int, VT_int=np.linalg.svd(X, full_matrices=True, compute_uv=True, hermitian=False)
        S_int=self.GAI_ETP(S_int,th,self.p)
        rank=len(np.where(S_int > self.eps)[0])
        if rank==0:
            rank=1  
        S_int=tf.compat.v1.matrix_diag(S_int) 
        Ak_int=tf.matmul(tf.matmul(U_int[:,0:rank],S_int[0:rank,0:rank]),VT_int[0:rank,:]) 
          
        return Ak_int    
 
    def Updata_E(self, Data, X, Lambda): 
        Temp= Data-X+Lambda/self.mu 
        return self.sparse_col(  Temp, self.lam/self.mu)
 

   

    def sparse_col(self, E, tau): 
        result=[]
        n1,n2=np.shape(E)
        for i in range(n2):
            result.append(self.GAI_ETP(tf.norm(E[:,i]),tau,self.p)*E[:,i]/(tf.norm(E[:,i])+self.eps)) 
        return tf.transpose(result,[1,0])

  
     

    def Updata_Lambda(self, Data, X, E, Lambda):
        Lambda= Lambda+tf.multiply(self.mu, Data-X-E)
        return Lambda
 

    def getLable(self, E,th): 
        Elabel=[]
        n1,n2=np.shape(E)
        for i in range(n2):
            if tf.norm(E[:,i])>th:
                Elabel.append(1)
            else:
                Elabel.append(0)
        return Elabel
         
        
   

    def GAI_ETP(self,xi,lambd,gamma,Iter=10,tol=10**(-6)): 
        a0=np.max([np.log((1-np.exp(-gamma))/(lambd*gamma**2))/(-gamma),0]) 
        delta=a0+self.ETP_gradient(a0,gamma,lambd) 
        if np.shape(xi) == ():
            x=[xi]
        else:
            x=xi
        n=len(x)
        y=[]
 

     
        for j in range(n):
            gradient_g= self.ETP_gradient(x[j],gamma,lambd)
            if gradient_g==0:
                BARx_b=x[j] 
            else:
                if delta<x[j]:
                    a=x[j]
                    sto=0
                    kkk=0
                    while sto==0:
                        kkk=kkk+1
                        a1=x[j]-self.ETP_gradient(a,gamma,lambd)
                        a2=x[j]-self.ETP_gradient(a1,gamma,lambd) 
                        if np.abs(a2-2*a1+a)<tol or kkk>=Iter:
                            if np.abs(a2-2*a1+a)<self.eps:
                                BARx_b=a2
                            else:
                                BARx_b=a1-(a2-a1)*(a1-a)/(a2-2*a1+a)
                            break
                        a=a1-(a2-a1)*(a1-a)/(a2-2*a1+a)  
                else:
                    BARx_b=a0
            if self.ETP_fun(BARx_b,x[j],gamma,lambd)<=self.ETP_fun(0,x[j],gamma,lambd):
                y.append(BARx_b)   
            else:
                y.append(0)
                    
        if np.shape(xi) == ():
            return np.array(y[0],np.float64) 
        else:
            return np.array(y,np.float64)
      
           
    #ETP's gradient 
    def ETP_gradient(self,x,gamma,lambd):
        gradient_g=gamma*lambd*np.exp(-gamma*x)/(1-np.exp(-gamma)) 
        return gradient_g
 
    def ETP_fun(self,x,y,gamma,lambd):
        g=1/2*(y-x)**2+lambd*(1-np.exp(-gamma*x))/(1-np.exp(-gamma))
        return g   



In [ ]:
# test sample
Data=np.array([[7,1,2],[5,2,4],[0,3,6]], np.float64)
n1,n2=np.shape(Data)
lam=6#1/np.sqrt(n1)
th=0.0000001
rho=1.5
mup=1.25
p=10 
HETP=OPETP_main(Data,lam,th,rho,mup,p)
print(HETP.Elabel)
print(HETP.FE)

Stop Criterion: 2.4261219471294536e-16
[1, 0, 0]
tf.Tensor(
[[ 7.00000000e+00 -0.00000000e+00  0.00000000e+00]
 [ 5.00000000e+00 -0.00000000e+00  0.00000000e+00]
 [-5.92118946e-15 -0.00000000e+00  0.00000000e+00]], shape=(3, 3), dtype=float64)


In [ ]:
# OP Model with LOG
from tensorflow.python.ops.linalg_ops import norm_v2
class OPLOG_main():
    
    def __init__(self, Data, lam=1/np.sqrt(n1),th=0.000000000000001, rho=1.5, mup=1.25,p=1,Maxiter=1000,tol=10**(-7) ):
        
         

        self.p=p
        n1,n2=np.shape(Data)
        self.lam=lam  
        self.eps=0.000000000000001
        self.d_nrom=norm_v2(Data)
        norm_two=np.linalg.svd(Data,compute_uv=False)[0] 
        norm_inf=norm_v2(Data,np.inf)/self.lam 
        dual_norm=np.max([norm_two, norm_inf])
        Lambda_int=Data/dual_norm 
        self.mu=mup/norm_two #1.25
        self.maxmu=self.mu*10**(7)
        self.rho=rho #1.5
        self.stages=Maxiter
        self.tol=tol
        
        X_int, E_int= tf.zeros([n1,n2], tf.float64), tf.zeros([n1,n2], tf.float64)
        value_sets=(Data, X_int, E_int, Lambda_int)
        self.FX, self.FE = self.OP_ADMM_main(value_sets) 
        self.Elabel=self.getLable(self.FE,th)
 
    def OP_ADMM_main(self, inputs):
        (Data, X_int, E_int, Lambda_int) = inputs  
        self.X,self.E,self.Lambda  = [],[],[]  
        self.E.append(self.Updata_E(Data, X_int, Lambda_int))  
        self.X.append(self.Updata_X(Data, self.E[-1], Lambda_int))      
        self.Lambda.append(self.Updata_Lambda(Data, self.X[-1], self.E[-1], Lambda_int))  
        self.mu=np.min([self.mu*self.rho, self.maxmu])  
        for stage in range(1,self.stages): 
            self.E.append(self.Updata_E(Data, self.X[-1], self.Lambda[-1]))  
            self.X.append(self.Updata_X(Data, self.E[-1], self.Lambda[-1]))      
            self.Lambda.append(self.Updata_Lambda(Data, self.X[-1], self.E[-1], self.Lambda[-1]))   
            self.mu=np.min([self.mu*self.rho, self.maxmu]) 
            stopCriterion= norm_v2(Data-self.X[-1]-self.E[-1])/self.d_nrom 
            print("Stop Criterion: {}".format(stopCriterion)) 
            if stopCriterion < self.tol:
                break 
        return self.X[-1], self.E[-1]   
     

     

    def Updata_X(self,Data, E, Lambda): 
        temp=-E+Data+Lambda/self.mu  
        return self.SVT(temp,1/self.mu)

 
   
    def SVT(self,X,th):
        n1, n2= np.shape(X)
        rank=np.min([n1,n2])  
        U_int, S_int, VT_int=np.linalg.svd(X, full_matrices=True, compute_uv=True, hermitian=False)
        S_int=self.GAI_LOG(S_int,th,self.p)
        rank=len(np.where(S_int > self.eps)[0])
        if rank==0:
            rank=1  
        S_int=tf.compat.v1.matrix_diag(S_int) 
        Ak_int=tf.matmul(tf.matmul(U_int[:,0:rank],S_int[0:rank,0:rank]),VT_int[0:rank,:]) 
          
        return Ak_int    
 
    def Updata_E(self, Data, X, Lambda):
        
        Temp= Data-X+Lambda/self.mu
        return self.sparse_col(  Temp, self.lam/self.mu)
 

   

    def sparse_col(self, E, tau): 
        result=[]
        n1,n2=np.shape(E)
        for i in range(n2):
            result.append(self.GAI_LOG(tf.norm(E[:,i]),tau,self.p)*E[:,i]/(tf.norm(E[:,i])+self.eps)) 
        return tf.transpose(result,[1,0])

  
     

    def Updata_Lambda(self, Data, X, E, Lambda):
        Lambda= Lambda+tf.multiply(self.mu, Data-X-E)
        return Lambda
 

    def getLable(self, E,th): 
        Elabel=[]
        n1,n2=np.shape(E)
        for i in range(n2):
            if tf.norm(E[:,i])>th:
                Elabel.append(1)
            else:
                Elabel.append(0)
        return Elabel
         

    def LOG_th(self,x,tau,lambd):
        x0=np.sqrt(2*lambd)-tau
        y=np.sign(np.max([x-x0,0]))*(1/2)*((x-tau)+np.sqrt((x+tau)**2-2*lambd))+np.sign(np.max([-x0-x,0]))*(1/2)*((x+tau)+np.sqrt((x-tau)**2-2*lambd))
        return y
    
    
    
    def GAI_LOG(self,xi,lambd,gamma,Iter=10,tol=10**(-4)):
        #Iter=10; 
    
        a0=np.max([np.sqrt(lambd)-gamma,0])
        #print(a0)
        delta=a0+self.LOG_gradient(a0,gamma,lambd)
        #print(delta) 
        if np.shape(xi) == ():
            x=[xi]
        else:
            x=xi
        n=len(x)
        y=[] 
     
        for j in range(n):
            gradient_g= self.LOG_gradient(x[j],gamma,lambd)
            if gradient_g==0:
                BARx_b=x[j] 
            else:
                if delta<x[j]:
                    a=x[j]
                    sto=0
                    kkk=0
                    while sto==0:
                        kkk=kkk+1
                        a1=x[j]-self.LOG_gradient(a,gamma,lambd)
                        a2=x[j]-self.LOG_gradient(a1,gamma,lambd) 
                        if np.abs(a2-2*a1+a)<tol or kkk>=Iter:
                            if np.abs(a2-2*a1+a)<self.eps:
                                BARx_b=a2
                            else:
                                BARx_b=a1-(a2-a1)*(a1-a)/(a2-2*a1+a)
                            break
                        a=a1-(a2-a1)*(a1-a)/(a2-2*a1+a) 
                        #print(a)
                else:
                    BARx_b=a0
                if self.LOG_fun(BARx_b,x[j],gamma,lambd)<=self.LOG_fun(0,x[j],gamma,lambd):
                    y.append(BARx_b)   
                else:
                    y.append(0)
        
        if np.shape(xi) == ():
            return np.array(y[0],np.float64) 
        else:
            return np.array(y,np.float64)
      
           


#LOG's gradient 
    def LOG_gradient(self,x,gamma,lambd):
        gradient_g=lambd/(gamma+x)
        return gradient_g
 
    def LOG_fun(self,x,y,gamma,lambd):
        g=1/2*(y-x)**2+lambd*(np.log(gamma+x))
        return g
 

In [ ]:
# test sample
Data=np.array([[7,1,2],[5,2,4],[0,3,6]], np.float64)
n1,n2=np.shape(Data)
lam=9#1/np.sqrt(n1)
th=0.0000001
rho=1.5
mup=1.25
p=23 
HLOG=OPLOG_main(Data,lam,th,rho,mup,p)
print(HLOG.Elabel)
print(HLOG.FE)

Stop Criterion: 0.0026366742250366285
Stop Criterion: 0.0016644579340530348
Stop Criterion: 0.000599125718678935
Stop Criterion: 0.0002322995954917029
Stop Criterion: 9.510577059916805e-05
Stop Criterion: 4.023698178748473e-05
Stop Criterion: 1.736866056257894e-05
Stop Criterion: 7.594570802103793e-06
Stop Criterion: 3.351016147898213e-06
Stop Criterion: 1.4909650435274126e-06
Stop Criterion: 4.1122749716615434e-07
Stop Criterion: 1.6144436485787669e-07
Stop Criterion: 7.165158124760496e-08
[1, 0, 0]
tf.Tensor(
[[ 1.51067052 -0.         -0.        ]
 [ 1.03186057  0.          0.        ]
 [-0.11010954  0.          0.        ]], shape=(3, 3), dtype=float64)
